# Visualization notebook



## Imports and functions

In [11]:
import utils
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import warnings
import spacy
from nltk import bigrams
from collections import Counter
warnings.filterwarnings("ignore", category=FutureWarning)
%matplotlib inline

# 1,071,477 authors

Counter({('This', 'is'): 1,
         ('is', 'a'): 1,
         ('a', 'NOUN'): 1,
         ('NOUN', 'and'): 1,
         ('and', 'I'): 1,
         ('I', 'VERB'): 1,
         ('VERB', 'NOUN'): 1,
         ('NOUN', 'NOUN'): 1,
         ('NOUN', 'the'): 1})

In [26]:
nlp = utils.load_spacy("en_core_web_md")


doc = nlp("This is a big string")





def pos_subsequences(doc):
    
    pos = [token.pos_ for token in doc]
    
    subseqs = []
    for i in range(len(pos)-1):
        subseqs.extend([(pos[i],pos[n]) for n in range(i+1, len(pos))])   
    return Counter(subseqs)
    
print([token.pos_ for token in doc])
print(pos_subsequences(doc))


['PRON', 'AUX', 'DET', 'ADJ', 'NOUN']
Counter({('PRON', 'AUX'): 1, ('PRON', 'DET'): 1, ('PRON', 'ADJ'): 1, ('PRON', 'NOUN'): 1, ('AUX', 'DET'): 1, ('AUX', 'ADJ'): 1, ('AUX', 'NOUN'): 1, ('DET', 'ADJ'): 1, ('DET', 'NOUN'): 1, ('ADJ', 'NOUN'): 1})


[2, 3, 1, 4, 2, 1, 1]

In [9]:
import ijson
import sqlite3
import random

data = ijson.parse(open("data/mud/full/raw_mud/raw_all/data.jsonl"), multiple_values=True)

i = 0
num_authors = 100

d = {}
posts = []
for prefix, event, value in data:
    
    if prefix == "syms.item":
        posts.append(value)
        
    if prefix.startswith("author_id"):
        d[value] = posts
        posts = []
        i += 1

    if i == num_authors:
        break



# author = random.choice(list(d.keys()))


# save = {author:d[author]}
# print(len(d[author]))

# utils.save_json(save, "example_author.json")



[(3, 4)]